In [1]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## Validating a Machine Learning Model

### Train-test split

We have already seen that when assessing the quality of a machine learning model (e.g., linear regression) it is usually not the best idea to use the same data as the ones that were used in the process of fitting (or *training*) the model. A fair assessment can only be obtain if the model is presented data it has never seen before. But where are we supposed to obtain such data if we have only one data set (e.g., Auto MPG)? Usually, we cannot just create new data on the fly -- the new data must be collected and the data collection process is probably the most expensive and time-consuming part of any data scientist job. So, what do we do if we are only limited to the data set at hand? One popular approach is to split the data set you are given into two chunks: 
  * a larger chunk including about 60-95% of the data which we will be used for training; it is called a *training set*;
  * a smaller chunk including about 5-40% of the data which will be used for assessing the quality of the model; it is called a *test* or *holdout set*.

  <img src='https://drive.google.com/uc?export=view&id=1sN5LobAApdl3pwqZBlioDNwnYlbkTsaI' width='800' align="center"/>

The idea here is that we must never use the test set in the process of fitting our model -- the model is not supposed to see these data during the training phase. Then checking the quality of the model on the test set will give us a much better estimate of the actual performance of the model. This should be done only at the very last stage of our work, when we are ready to deploy the model.

In sklearn, we can easily create a split into the training set and test set using a special function called `train_test_split()`. This function takes your features and labels as an input and spits out the splitted versions of those. Here is an example illustrating how this function works on the Auto MPG data (we are only using the car MPG and weight data):

In [2]:
import numpy as np
from pathlib import Path
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
from sklearn.model_selection import train_test_split

PATH = Path('/content/gdrive/My Drive/Colab Notebooks/Applied_Machine_Learning/Data/Auto_MPG')

weight=np.load(PATH/'weight.npy')
mpg=np.load(PATH/'mpg.npy')

X = weight.reshape(-1, 1)
y = mpg

print(f"The shape of the 'weight' data is {X.shape}.")
print(f"The shape of the 'mpg' data is {y.shape}.")

# By default, 'train_test_split' will perform random shuffling.
# To get a reproducible split, set the value of the 'random_state' parameter.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1442)

print(f"The shape of X_train is {X_train.shape}.")
print(f"The shape of y_train is {y_train.shape}.")
print(f"The shape of X_test is {X_test.shape}.")
print(f"The shape of y_test is {y_test.shape}.")

The shape of the 'weight' data is (398, 1).
The shape of the 'mpg' data is (398,).
The shape of X_train is (278, 1).
The shape of y_train is (278,).
The shape of X_test is (120, 1).
The shape of y_test is (120,).


### K-fold cross-validation

Once we have our data split into the test set and training set, we become ready to fit our model on the training data. But what model are we supposed to use? We know that the relationship between our label (MPG) and feature (weight) is non-linear but we don't really know what power would work best. We also do not know what set of hyperparameters (e.g., the learning rate $\eta$, regularization term coefficient $\alpha$) would be optimal. 

To address all these questions we need to run a lot of experiments with our training data trying various polynomial models with different hyperparameters. To decide which one of these experiments yields the best result we need to be able to compare their outcomes, for example, by computing and comparing the resulting $\text{RMSE}$ scores (the metric). But here we have the same problem as before: to be able to make an objective comarisson we need to compute the $\text{RMSE}$ scores on data that have never been shown to the model during its training phase. We cannot use the test set data -- those are supposed to be used only at the very last stage when we are ready to deploy our model. So what do we do?

One solution would be to use a chunk of our training data for the on-going quality assessment of different models. This chunk is typically called the *validation set* or *development set* and it can be created using the same `train_test_split()` function as before. 

  <img src='https://drive.google.com/uc?export=view&id=1sQc3ojQj4WhZvQ6efuRR5yk2O-0xNa1E' width='800' align="center"/>

You may have noticed that this validation set method does have some downsides. For example, in order for this method to work, we need to exclude some part of the data from the training process. Thus, the model is going to be exposed to a smaller amount of data during the training phase and, as a result, may demonstrate a worse performance. We have a dilemma: on one hand, if the data set is split into the training and validation sets then we can get an objective assessment of the performance of our models; but then the performance of the models may get worse; on the other hand, if we do not do the split then we are loosing a way to objectively compare performances of different models. So, what do we do? One popular solution to this dilemma is to use a *K-fold cross-validation* method in place of the train-test split method. 

The idea of K-fold cross-validation is illustrated in the picture below (with $K=5$). 

<img src='https://drive.google.com/uc?export=view&id=1sTHRRLnZzX6dGwM2xxLwJByA7ANJnpNx' width='600' align="center"/>

We split the training set into $K$ non-overlapping folds (a very popular choice is $K=5$). And then we train our model not one but $K$ times: one time for each fold; every time we use the current fold data for validation and the rest of the data for training. Essentially, at the end of this process, we end up with $K$ slightly different models, trained on slightly different data. The overall metric is computed as an average of the $K$ metric values computed for individual folds and, when making predictions on new data, we again just average the individual prediction of the $K$ models. Note that during the training process all training data are used (though not simultaneously), so we can expect a better performance that the one we get with a simple validation set approach.

A great news is that sklearn can automate this process for us with the help of a special `cross_val_score()` function. Here is an example illustrating the usage of this function (we are using the same car MPG and car weight data as before).

In [3]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso

n_folds=5
poly_degree=2
alpha=0.05

cv_pipe = Pipeline([                    
                    ('poly', PolynomialFeatures(degree=poly_degree, include_bias=False)),
                    ('scaler', StandardScaler()),
                    ('lasso', Lasso(alpha=alpha))
                    ])

score = cross_val_score(cv_pipe, X=X_train, y=y_train, cv=n_folds, scoring='neg_root_mean_squared_error')

print(f"The RMSE scores for the folds:\n {-score}\n")
print(f"The average over {n_folds} folds RMSE score is {np.mean(-score):.3f}.")

The RMSE scores for the folds:
 [4.54200692 4.77359256 3.58901793 4.54451504 4.38312039]

The average over 5 folds RMSE score is 4.366.


Another popular regression metric is the $R^2$ score, or the *coefficient of determination* which is a measure of how well our model performs relative to a simple mean of the target values. The highest and the best value for $R^2$ is $1$ -- it means that the model reproduces all data points perfectly; $R^2=0$ means that the model performs not better that a simple avearge of the target variable; $R^2 < 0$ means that the model performs even worse than a simple average.

In [4]:
score_r2 = cross_val_score(cv_pipe, X=X_train, y=y_train, cv=n_folds, scoring='r2')

print(f"The R^2 scores for the folds:\n {score_r2}\n")
print(f"The average over {n_folds} folds R^2 score is {np.mean(score_r2):.3f}.")

The R^2 scores for the folds:
 [0.69329691 0.69215281 0.74066015 0.72801033 0.65797403]

The average over 5 folds R^2 score is 0.702.


### Grid search

Sklearn provides a convenient way to search for the best parameters of a model using `GridSearchCV()` class. Here is an example:

In [5]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
from sklearn.model_selection import GridSearchCV

# make lists of different parameters to check
parameters = [
              {'poly__degree':[2, 3, 4], 'lasso__alpha': [0.5, 0.05, 0.005]},
              {'poly__degree':[2, 3, 4]}
              ]
# Instantiate
grid_pipeline = GridSearchCV(cv_pipe, parameters, cv=n_folds, scoring='neg_root_mean_squared_error')
# Fit
grid_pipeline.fit(X_train,y_train)
# Show results
grid_pipeline.cv_results_

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.04367864565711, tolerance: 1.4898354170403587
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 32.82796465639876, tolerance: 1.425937211711712
  positive)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 5.375957804428253, tolerance: 1.4898354170403587
  positive)


{'mean_fit_time': array([0.00195899, 0.00244994, 0.00146036, 0.00146465, 0.00142345,
        0.00123806, 0.00133982, 0.00178108, 0.00318127, 0.00184565,
        0.0026032 , 0.0022202 ]),
 'mean_score_time': array([0.00072508, 0.00090504, 0.00061727, 0.00057302, 0.00054326,
        0.00052156, 0.00052853, 0.00055895, 0.00076571, 0.00080943,
        0.00108638, 0.00095644]),
 'mean_test_score': array([-4.53478109, -4.53478109, -4.53478109, -4.36645057, -4.31468616,
        -4.30832062, -4.29836053, -4.29753114, -4.30284297, -4.36645057,
        -4.31468616, -4.30832062]),
 'param_lasso__alpha': masked_array(data=[0.5, 0.5, 0.5, 0.05, 0.05, 0.05, 0.005, 0.005, 0.005,
                    --, --, --],
              mask=[False, False, False, False, False, False, False, False,
                    False,  True,  True,  True],
        fill_value='?',
             dtype=object),
 'param_poly__degree': masked_array(data=[2, 3, 4, 2, 3, 4, 2, 3, 4, 2, 3, 4],
              mask=[False, False, Fals

In [6]:
# Printing the best parameters
grid_pipeline.best_params_

{'lasso__alpha': 0.005, 'poly__degree': 3}

In [7]:
# Call predict on the estimator with the best found parameters
y_pred=grid_pipeline.predict(X_train)

In [8]:
from sklearn.metrics import mean_squared_error, r2_score

grid_rmse=mean_squared_error(y_train, y_pred)**0.5
print(f"The RMSE score after grid search CV is {grid_rmse:.3f}")

grid_r2=r2_score(y_train, y_pred)
print(f"The R^2 score after grid search CV is {grid_r2:.3f}")

The RMSE score after grid search CV is 4.272
The R^2 score after grid search CV is 0.719
